use loc[i, field] instead of iloc[i, field], iloc is very slow.

In [1]:
import sys
sys.path = ['/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/RecStudio/'] + sys.path
import os
import random
import numpy as np
import pandas as pd
import cudf, itertools
import scipy.sparse as ssp
from functools import lru_cache, partial
from tqdm import tqdm, trange
from collections import Counter, defaultdict
import torch
import pickle


In [2]:
def cast_dtype(df : pd.DataFrame, columns=None):
    if columns is None:
        columns = df.columns
    for k in columns:
        dt = type(df[k].iloc[0])
        if 'float' in str(dt):
            df[k] = df[k].astype('float32')
        elif 'int' in str(dt):
            df[k] = df[k].astype('int32')
        elif dt == list:
            dt_ = type(df.iloc[0][k][0])
            if 'float' in str(dt_):
                df[k] = df[k].apply(lambda x : np.array(x, dtype=np.float32))
            elif 'int' in str(dt_):
                df[k] = df[k].apply(lambda x : np.array(x, dtype=np.int32))

In [3]:
def _load_cache(path):
    with open(path, 'rb') as f:
        download_obj = pickle.load(f)
    return download_obj

In [4]:
def get_scores(merged_candidates_df, query_embeddings, product_embeddings):
    batch_size = 5096
    num_iter = (len(merged_candidates_df) - 1) // batch_size + 1
    score_list = []
    with torch.no_grad():
        for i in tqdm(range(num_iter)):
            st, ed = i * batch_size, (i + 1) * batch_size 
            batch_sess = merged_candidates_df.iloc[st : ed]
            batch_sess_id = torch.tensor(batch_sess['sess_id'].tolist(), dtype=torch.long, device=query_embeddings.device)
            batch_product_id = torch.tensor(batch_sess['dataset_id'].tolist(), dtype=torch.long, device=product_embeddings.device)
            query_emb = query_embeddings[batch_sess_id]
            product_emb = product_embeddings[batch_product_id]
            batch_score = (query_emb * product_emb).sum(dim=-1) 
            score_list.append(batch_score.cpu())
        score_list = torch.cat(score_list, dim=0).cpu().tolist()
        return score_list 

In [5]:
def normalize_scores(score_df, score_name, normalized_score_name):
    # score_df_g = cudf.from_pandas(score_df)
    score_df['exp_score'] = np.exp(score_df[score_name].to_numpy())
    scores_sum = score_df[['sess_id', 'exp_score']].groupby('sess_id').sum()
    scores_sum.reset_index(inplace=True)
    scores_sum = scores_sum.sort_values(by=['sess_id'], ascending=True)
    scores_sum.reset_index(drop=True, inplace=True)
    scores_sum.rename(columns={'exp_score' : 'score_sum'}, inplace=True)

    merged_score_df = score_df.merge(scores_sum, how='left', left_on=['sess_id'], right_on=['sess_id'])
    merged_score_df = merged_score_df.sort_values(by=['sess_id', 'product'])
    merged_score_df.reset_index(drop=True, inplace=True)
    
    # merged_score_df = merged_score_df_g.to_pandas(merged_score_df_g)
    score_df[normalized_score_name] = merged_score_df['exp_score'] / merged_score_df['score_sum']
    score_df['exp_score'] = merged_score_df['exp_score']
    score_df['score_sum'] = merged_score_df['score_sum']

    # del scores_sum_g
    # del merged_score_df_g 

# Merge test score

In [6]:
merged_candidates_feature_test_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/XGBoost/candidates_phase2/merged_candidates_150_test_feature.parquet'
valid_sessions_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/data_for_recstudio/task1_data/task13_4_task1_valid_sessions_phase2.csv'
test_sessions_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/raw_data/sessions_test_task1_phase2.csv'

In [7]:
@lru_cache(maxsize=1)
def read_merged_candidates_feature_test():
    return pd.read_parquet(merged_candidates_feature_test_path, engine='pyarrow')

@lru_cache(maxsize=1)
def read_valid_sessions():
    return pd.read_csv(valid_sessions_path)

@lru_cache(maxsize=1)
def read_test_sessions():
    return pd.read_csv(test_sessions_path)

In [8]:
FIELD_NAME = 'title_bert_scores'

In [9]:
DE_product_embeddings_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/text_method/phase2_task1_bert_title_results_DE/results/item_reps/reordered_item.npy'
DE_test_embeddings_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/text_method/phase2_task1_bert_title_results_DE/test_results/test_query_reps/query.npy'
JP_product_embeddings_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/text_method/phase2_task1_bert_title_results_JP/results/item_reps/reordered_item.npy'
JP_test_embeddings_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/text_method/phase2_task1_bert_title_results_JP/test_results/test_query_reps/query.npy'
UK_product_embeddings_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/text_method/phase2_task1_roberta_title_results_UK/results/item_reps/reordered_item.npy'
UK_test_embeddings_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/text_method/phase2_task1_roberta_title_results_UK/test_results/test_query_reps/query.npy'

In [10]:
DE_dataset_cache = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/.recstudio/cache/87c62409540df6ccca9d90ab244af0e5'
JP_dataset_cache = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/.recstudio/cache/d296613e4d5aa97bebf6c4b114f02d89'
UK_dataset_cache = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/.recstudio/cache/d3540d1aadb28b19da92e77c7cf0f7e2'

In [11]:
DE_train_dataset, DE_valid_dataset = _load_cache(DE_dataset_cache)
JP_train_dataset, JP_valid_dataset = _load_cache(JP_dataset_cache)
UK_train_dataset, UK_valid_dataset = _load_cache(UK_dataset_cache)
locale_map = {
    'DE' : DE_train_dataset.field2token2idx['product_id'], 
    'JP' : JP_train_dataset.field2token2idx['product_id'], 
    'UK' : UK_train_dataset.field2token2idx['product_id']
    }

In [12]:
merged_candidates = read_merged_candidates_feature_test()
test_sessions = read_test_sessions()
EMBED_DIM = 768
merged_candidates.sort_values(by=['sess_id', 'product'], inplace=True)
merged_candidates.reset_index(drop=True, inplace=True)

In [14]:
# sess embeddings 
test_DE_query_emb = torch.from_numpy(np.load(DE_test_embeddings_path)) 
test_JP_query_emb = torch.from_numpy(np.load(JP_test_embeddings_path))
test_UK_query_emb = torch.from_numpy(np.load(UK_test_embeddings_path))
test_query_embeddings = torch.empty(len(test_sessions), EMBED_DIM)
test_query_embeddings[(test_sessions[test_sessions['locale'] == 'DE'].index).tolist()] = test_DE_query_emb
test_query_embeddings[(test_sessions[test_sessions['locale'] == 'JP'].index).tolist()] = test_JP_query_emb
test_query_embeddings[(test_sessions[test_sessions['locale'] == 'UK'].index).tolist()] = test_UK_query_emb

In [15]:
test_query_embeddings, test_query_embeddings.shape

(tensor([[ 0.1136, -0.2501, -0.0704,  ...,  0.1847,  0.0717, -0.3017],
         [-0.3964,  0.0939, -0.1283,  ...,  0.1563, -0.1644, -0.0321],
         [-0.0323,  0.3606, -0.0128,  ..., -0.0118,  0.1365, -0.2942],
         ...,
         [ 0.0118,  0.5020, -0.0754,  ..., -0.4015,  0.0860,  0.0765],
         [ 0.0480,  0.0737, -0.1268,  ..., -0.1232,  0.1216,  0.1034],
         [ 0.2913,  0.2383, -0.2621,  ..., -0.2491,  0.0703,  0.0205]]),
 torch.Size([316972, 768]))

In [16]:
# product_embeddings, the embeddings include padding embedding
DE_product_emb = torch.from_numpy(np.load(DE_product_embeddings_path))
JP_product_emb = torch.from_numpy(np.load(JP_product_embeddings_path))
UK_product_emb = torch.from_numpy(np.load(UK_product_embeddings_path))
product_embeddings = torch.cat([DE_product_emb, JP_product_emb, UK_product_emb], dim=0)

In [17]:
DE_product_emb

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.2777,  0.4617,  0.1657,  ...,  0.0505,  0.0628, -0.7108],
        [-0.0915,  0.2569, -0.1503,  ..., -0.0563,  0.1256, -0.4411],
        ...,
        [ 0.2559,  0.1071, -0.1689,  ..., -0.0935, -0.0517, -0.4844],
        [-0.1851,  0.6337,  0.0548,  ...,  0.1978, -0.4778, -0.8956],
        [-0.1200,  0.1002, -0.4467,  ...,  0.1571, -0.4825, -0.3066]],
       dtype=torch.float64)

In [18]:
merged_candidates_ = merged_candidates[['sess_id', 'sess_locale', 'product']].copy()

In [19]:
DE_product_list, DE_id_list = list(zip(*locale_map['DE'].items()))
JP_product_list, JP_id_list = list(zip(*locale_map['JP'].items()))
UK_product_list, UK_id_list = list(zip(*locale_map['UK'].items()))
product_list = list(DE_product_list) + list(JP_product_list) + list(UK_product_list)
id_list = list(DE_id_list) + list(JP_id_list) + list(UK_id_list)
locale_list = ['DE'] * len(DE_id_list) + ['JP'] * len(JP_id_list) + ['UK'] * len(UK_id_list)
product_id_df = pd.DataFrame({'locale' : locale_list, 'product' : product_list, 'dataset_id' : id_list})

In [17]:
# merged_candidates_g = cudf.from_pandas(merged_candidates_)
# product_id_df_g = cudf.from_pandas(product_id_df)

In [18]:
# merged_candidates_score_g = merged_candidates_g.merge(product_id_df_g, how='left', left_on=['sess_locale', 'product'], right_on=['locale', 'product'])
# merged_candidates_score_g['dataset_id'] = merged_candidates_score_g['dataset_id'].fillna(0)
# merged_candidates_score_g.drop(columns=['locale'], inplace=True)
# merged_candidates_score_g = merged_candidates_score_g.sort_values(by=['sess_id', 'product'])
# merged_candidates_score_g.reset_index(drop=True, inplace=True)
# merged_candidates_score = merged_candidates_score_g.to_pandas()

In [20]:
merged_candidates_score = merged_candidates_.merge(product_id_df, how='left', left_on=['sess_locale', 'product'], right_on=['locale', 'product'])
merged_candidates_score['dataset_id'] = merged_candidates_score['dataset_id'].fillna(0)
merged_candidates_score.drop(columns=['locale'], inplace=True)
merged_candidates_score = merged_candidates_score.sort_values(by=['sess_id', 'product'])
merged_candidates_score.reset_index(drop=True, inplace=True)
assert len(merged_candidates_score) == len(merged_candidates)

In [19]:
# del merged_candidates_g
# del product_id_df_g
# del merged_candidates_score_g

In [21]:
locale_offset = {'DE' : 0, 'JP' : len(DE_product_list), 'UK' : len(DE_product_list) + len(JP_product_list)}
for locale in ['DE', 'JP', 'UK']:
    merged_candidates_score['dataset_id'][merged_candidates_score['sess_locale'] == locale] = \
        merged_candidates_score['dataset_id'][merged_candidates_score['sess_locale'] == locale] + locale_offset[locale]

/tmp/ipykernel_479727/2978421209.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_candidates_score['dataset_id'][merged_candidates_score['sess_locale'] == locale] = \


In [ ]:
valid_query_embeddings = test_query_embeddings.to('cuda:7')
product_embeddings = product_embeddings.to('cuda:7')

In [22]:
# using cosine scores 
normalized_test_query_embeddings = torch.nn.functional.normalize(test_query_embeddings, p=2, dim=-1)
normalized_product_embeddings = torch.nn.functional.normalize(product_embeddings, p=2, dim=-1)
normalized_product_embeddings = normalized_product_embeddings.type(torch.float)

In [23]:
merged_candidates_score['cos_'+FIELD_NAME] = get_scores(merged_candidates_score, normalized_test_query_embeddings, normalized_product_embeddings)

  0%|          | 0/32208 [00:00<?, ?it/s]/tmp/ipykernel_479727/622120297.py:10: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  batch_product_id = torch.tensor(batch_sess['dataset_id'].tolist(), dtype=torch.long, device=product_embeddings.device)
100%|██████████| 32208/32208 [12:25<00:00, 43.23it/s]


In [29]:
merged_candidates_score[FIELD_NAME] = get_scores(merged_candidates_score, test_query_embeddings, product_embeddings)

  0%|          | 0/12944 [00:00<?, ?it/s]/tmp/ipykernel_479727/1592792540.py:10: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  batch_product_id = torch.tensor(batch_sess['dataset_id'].tolist(), dtype=torch.long, device=product_embeddings.device)
100%|██████████| 12944/12944 [08:12<00:00, 26.26it/s]


In [36]:
# del test_query_embeddings
# del product_embeddings

In [30]:
normalize_scores(merged_candidates_score, FIELD_NAME, 'normalized_'+FIELD_NAME)

In [31]:
merged_candidates['cos_'+FIELD_NAME] = merged_candidates_score['cos_'+FIELD_NAME]

In [32]:
merged_candidates[FIELD_NAME] = merged_candidates_score[FIELD_NAME]
merged_candidates['normalized_'+FIELD_NAME] = merged_candidates_score['normalized_'+FIELD_NAME]

In [37]:
cast_dtype(merged_candidates, ['cos_'+FIELD_NAME])

In [38]:
cast_dtype(merged_candidates, [FIELD_NAME, 'normalized_'+FIELD_NAME])
merged_candidates.to_parquet(merged_candidates_feature_test_path, engine='pyarrow')

In [30]:
# verify scores
merged_candidates[merged_candidates['sess_id'] == 54216].sort_values(by=['normalized_sasrec_scores_2'], ascending=False)[['sess_locale', 'product', 'normalized_sasrec_scores_2', 'sasrec_scores_2', 'normalized_'+FIELD_NAME, FIELD_NAME, 'cos_'+FIELD_NAME]].iloc[:25]

,sess_locale,product,normalized_sasrec_scores_2,sasrec_scores_2,normalized_title_bert_scores,title_bert_scores,cos_title_bert_scores
16280933,DE,B01NAQD0KD,0.192159,16.101398,0.179538,388.531311,0.970479
16280917,DE,B00IVEPHQ4,0.171488,15.987590,0.005924,385.119873,0.962715
16280961,DE,B07RMTZKXY,0.138671,15.775178,0.002676,384.325226,0.964076
16280915,DE,B00E3862DE,0.116764,15.603232,0.021343,386.401642,0.962342
16280955,DE,B07M8J529S,0.045633,14.663700,0.000003,377.373230,0.943159
16280966,DE,B07Y53N2GW,0.044587,14.640528,0.062468,387.475555,0.968654
16281051,DE,B09FRW5J7B,0.036596,14.443015,0.024402,386.535583,0.968574
16281174,DE,B0BDRZ8TMB,0.031278,14.285977,0.001663,383.849487,0.963565
16281152,DE,B0B63375WT,0.024604,14.046004,0.018637,386.266083,0.966392
16281000,DE,B08JYFQ9ZH,0.022332,13.949091,0.002435,384.230988,0.965461
